In [ ]:
from google.colab import drive
path = drive.mount('/content/drive/')

In [ ]:
import csv
import collections
import numpy as np
from math import sqrt
import matplotlib.pyplot as plt
from sklearn import metrics
import pandas as pd

In [ ]:
def get_data(file_name):
  with open(file_name, newline='') as csvfile:
    content = csv.reader(csvfile)
    line_count = 0
    for row in content:
      if line_count > 0:
        data = [float(row[0]), float(row[1]), float(row[2]), float(row[3]), float(row[4]), float(row[5]), float(row[6])]
        all_data.append(data)
      
      line_count += 1  

In [ ]:
all_data = []
get_data(r'/content/drive/MyDrive/Capstone Material/Code/final_train_data.csv')
data = np.array(all_data)
print(data.shape)
np.random.shuffle(data)

training_data = []
training_label = []

for d in data:
  training_data.append([d[2], d[3], d[4], d[5], d[6]])
  training_label.append([d[0], d[1]])

(2656, 7)


In [ ]:
all_data = []
get_data(r'/content/drive/MyDrive/Capstone Material/Code/final_test_data.csv')
data = np.array(all_data)
print(data.shape)
np.random.shuffle(data)

testing_data = []
testing_label = []

for d in data:
  if [d[2], d[3], d[4]] not in training_data:
    testing_data.append([d[2], d[3], d[4], d[5], d[6]])
    testing_label.append([d[0], d[1]])

print(len(testing_data))

(592, 7)
592


In [ ]:
# calculate difference between powers (db)
def difference(test_values, train_values):
	distance = 0.0
	for i in range(len(test_values)):
		distance += (test_values[i] - train_values[i])**2
	return sqrt(distance)

# Find the most similar neighbors
def find_neighbors(train, test_values, num_neighbors):
	distances = list()
	for i in range(len(train)):
		diff = difference(test_values, train[i])
		distances.append((i, diff))
	distances.sort(key=lambda val: val[1])
 
	neighbors = list()
	for i in range(num_neighbors):
		neighbors.append(distances[i][0])
	return neighbors 
 

In [ ]:
# exponential prediction
def exponential_calc(values, alpha):
  numerator = 0
  denominator = 0
  for i in range(len(values)):
    if i == 0:
      numerator += values[i]
      denominator += 1
    else:
      numerator += ((1-alpha)**(2*i))*values[i]
      denominator += (1-alpha)**(2*i)

  return numerator/denominator

def predict(training_data, testing_data):
  res = []
  alpha = 2/10
  for i in range(len(testing_data)):
   neighbors = find_neighbors(training_data, testing_data[i], 10)
   train_x = []
   train_y = []
   for neighbor in neighbors:
      train_x.append(training_label[neighbor][0])
      train_y.append(training_label[neighbor][1])
   
   final_x = exponential_calc(train_x,alpha)
   final_y = exponential_calc(train_y,alpha)
   res.append([final_x, final_y])
  
  r = metrics.mean_absolute_error(testing_label, res)
  print(r)

predict(training_data, testing_data)

In [ ]:
def predict(training_data, testing_data):

	res = []

	for i in range(len(testing_data)):
		neighbors = find_neighbors(training_data, testing_data[i], 10)

		avg_x = 0.0
		avg_y = 0.0
		for neighbor in neighbors:
			avg_x += training_label[neighbor][0]
			avg_y += training_label[neighbor][1]
		
		avg_x = avg_x/10
		avg_y = avg_y/10

		res.append([avg_x, avg_y])
	
	r = metrics.mean_absolute_error(testing_label, res)
	print(r)

predict(training_data, testing_data)